<a href="https://colab.research.google.com/github/e19166/e19-4yp-Dynamic-Multi-Dimensional-Resource-Orchestration-in-Kubernetes/blob/main/Models/mlp/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install river optuna pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 whi

# Service 1

In [3]:
import pandas as pd
import numpy as np
from river import neural_net, metrics, preprocessing, compose
from river.neural_net import MLPRegressor
import optuna

# Load data
df = pd.read_csv("/content/service-1-deployment_dataset.csv")
df['Timestamp'] = pd.to_datetime(df['Timestamp'], format = 'mixed')
df = df.sort_values("Timestamp")

# Features
df['cpu_usage_pct'] = df['CPU Usage'] / df['CPU Limit']
df['memory_usage_pct'] = df['Memory Usage'] / df['Memory Limit']
df['request_rate_rps'] = df['Request Rate']
df['cpu_allocated'] = df['CPU Request']
df['memory_allocated'] = df['Memory Request']
df['hour'] = df['Timestamp'].dt.hour
df['dayofweek'] = df['Timestamp'].dt.dayofweek

# Rolling stats
df['cpu_usage_mean_5'] = df['cpu_usage_pct'].rolling(window=5).mean().bfill()
df['memory_usage_mean_5'] = df['memory_usage_pct'].rolling(window=5).mean().bfill()
df['cpu_usage_std_5'] = df['cpu_usage_pct'].rolling(window=5).std().bfill()
df['memory_usage_std_5'] = df['memory_usage_pct'].rolling(window=5).std().bfill()

# Targets
df['cpu_needed_t+1'] = df['CPU Usage'].shift(-1)
df['memory_needed_t+1'] = df['Memory Usage'].shift(-1)
df = df.dropna()


In [4]:
features = [
    'cpu_usage_pct', 'memory_usage_pct', 'request_rate_rps',
    'cpu_allocated', 'memory_allocated',
    'cpu_usage_mean_5', 'memory_usage_mean_5',
    'cpu_usage_std_5', 'memory_usage_std_5',
    'hour', 'dayofweek'
]

X = df[features].to_dict(orient='records')
y_cpu = df['cpu_needed_t+1'].values
y_mem = df['memory_needed_t+1'].values


In [7]:
def evaluate_nn(params):
    cpu_model = compose.Pipeline(
        preprocessing.StandardScaler(),
        MLPRegressor(
            layers=(params['hidden_size'],),    # Convert int to tuple here
            learning_rate=params['learning_rate'],
            optimizer=params['optimizer'],
            activation='relu'
        )
    )

    mem_model = compose.Pipeline(
        preprocessing.StandardScaler(),
        MLPRegressor(
            layers=(params['hidden_size'],),
            learning_rate=params['learning_rate'],
            optimizer=params['optimizer'],
            activation='relu'
        )
    )
    mae_cpu = metrics.MAE()
    r2_cpu = metrics.R2()
    mae_mem = metrics.MAE()
    r2_mem = metrics.R2()

    for xi, y_cpu_i, y_mem_i in zip(X, y_cpu, y_mem):
        pred_cpu = cpu_model.predict_one(xi)
        pred_mem = mem_model.predict_one(xi)

        if pred_cpu is not None:
            mae_cpu.update(y_cpu_i, pred_cpu)
            r2_cpu.update(y_cpu_i, pred_cpu)
        if pred_mem is not None:
            mae_mem.update(y_mem_i, pred_mem)
            r2_mem.update(y_mem_i, pred_mem)

        cpu_model.learn_one(xi, y_cpu_i)
        mem_model.learn_one(xi, y_mem_i)

    return mae_cpu.get(), r2_cpu.get(), mae_mem.get(), r2_mem.get()


In [6]:
def objective(trial):
    hidden_size = trial.suggest_int("hidden_size", 5, 100)
    learning_rate = trial.suggest_float("learning_rate", 1e-4, 0.1, log=True)
    optimizer = trial.suggest_categorical("optimizer", ["adam", "sgd", "rmsprop"])

    params = {
        "hidden_layers": (hidden_size,),
        "learning_rate": learning_rate,
        "optimizer": optimizer
    }

    mae_cpu, _, mae_mem, _ = evaluate_nn(params)
    return mae_cpu + mae_mem  # Minimize total MAE

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

print("Best parameters:", study.best_params)


[I 2025-06-30 03:13:13,857] A new study created in memory with name: no-name-62a158aa-88fe-4edb-b391-9ec10b625a11
[W 2025-06-30 03:13:13,860] Trial 0 failed with parameters: {'hidden_size': 96, 'learning_rate': 0.047691732809878686, 'optimizer': 'sgd'} because of the following error: TypeError("MLPRegressor.__init__() got an unexpected keyword argument 'hidden_layers'").
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 201, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipython-input-6-1739792022.py", line 12, in objective
    mae_cpu, _, mae_mem, _ = evaluate_nn(params)
                             ^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-5-563712252.py", line 4, in evaluate_nn
    MLPRegressor(
TypeError: MLPRegressor.__init__() got an unexpected keyword argument 'hidden_layers'
[W 2025-06-30 03:13:13,863] Trial 0 failed with value None.


TypeError: MLPRegressor.__init__() got an unexpected keyword argument 'hidden_layers'